In [1]:
cd ~/bench

/home/soda/rcappuzz/work/benchmark-join-suggestions


This notebook is used to combine all results from various runs in to a single file for simplicity and better storage. 

In [2]:
import polars as pl
from pathlib import Path
from src.utils.logging import read_and_process, read_logs

In [3]:
cfg = pl.Config()
cfg.set_fmt_str_lengths(150)

polars.config.Config

In [4]:
dest_path = Path("results/overall")
overall_list = []

# Wordnet 3K

In [5]:
r_path = "results/logs/0424-xc16vll9"
df_raw = read_logs(exp_name=None, exp_path=r_path)
df_raw = df_raw.fill_null(0).with_columns(
    pl.lit(0.0).alias("auc"), pl.lit(0.0).alias("f1score")
)
overall_list.append(df_raw)

In [6]:
df_raw.write_parquet(Path(dest_path, "wordnet-3k_first.parquet"))

# Wordnet 10k

In [7]:
r_path = "results/logs/0428-hde9c7an"
df_raw = read_logs(exp_name=None, exp_path=r_path)
df_raw = df_raw.fill_null(0).with_columns(
    pl.lit(0.0).alias("auc"), pl.lit(0.0).alias("f1score")
)
overall_list.append(df_raw)

In [8]:
df_raw.write_parquet(Path(dest_path, "wordnet-10k_first.parquet"))

# Wordnet old + Binary

In [9]:
r_path = "results/logs/0429-oot6vo5b"
df_raw = read_logs(exp_name=None, exp_path=r_path)
df_raw = df_raw.fill_null(0).with_columns(
    pl.lit(0.0).alias("auc"), pl.lit(0.0).alias("f1score")
)
overall_list.append(df_raw)

In [10]:
df_raw.write_parquet(Path(dest_path, "old-versions_first.parquet"))

# Wordnet 50k

In [11]:
r_path = "results/logs/0430-e6fv2b39"
df_raw = read_logs(exp_name=None, exp_path=r_path)
df_raw = df_raw.fill_null(0).with_columns(
    pl.lit(0.0).alias("auc"), pl.lit(0.0).alias("f1score")
)
overall_list.append(df_raw)

In [12]:
df_raw.write_parquet(Path(dest_path, "wordnet-50k_first.parquet"))

In [13]:
for df in overall_list:
    print(df.schema)
df_overall = pl.concat(overall_list)
df_overall.write_parquet(Path(dest_path, "overall_first.parquet"))

OrderedDict([('scenario_id', Int64), ('status', String), ('target_dl', String), ('jd_method', String), ('base_table', String), ('query_column', String), ('estimator', String), ('aggregation', String), ('chosen_model', String), ('fold_id', Int64), ('time_fit', Float64), ('time_predict', Float64), ('time_run', Float64), ('time_prepare', Float64), ('time_model_train', Float64), ('time_join_train', Float64), ('time_model_predict', Float64), ('time_join_predict', Float64), ('peak_fit', Float64), ('peak_predict', Float64), ('peak_test', Float64), ('r2score', Float64), ('rmse', Float64), ('f1score', Float64), ('auc', Float64), ('n_cols', String), ('budget_type', String), ('budget_amount', Int64), ('epsilon', Float64)])
OrderedDict([('scenario_id', Int64), ('status', String), ('target_dl', String), ('jd_method', String), ('base_table', String), ('query_column', String), ('estimator', String), ('aggregation', String), ('chosen_model', String), ('fold_id', Int64), ('time_fit', Float64), ('time_p